# TODO

- [X] Use the training set to select the best teacher --> no significant improvement

- [X] Test when nodes have "small data" (reproduce Pablo's results) --> After a while, decentralised learning starts degrading the performance (as in the Pablo's case). 

- [ ] Why distillation stops learning after the first few rounds

- [ ] Add other FedL as benchmarks

- [ ] Probabilistic selection of the teacher model

- [X] Probabilitic selection between 'follow the best teacher' and 'FedAvg' --> apparent degradation (with p=0.5); 

- [ ] Probabilistic model fw

- [X] Test other graph topologies: done with barabasi and erdos-renyi

- [X] Check the "distance" between distilled models, federated model and decentralised models

- [ ] Test how many neight you need for being resilient from the averaging model disruption


 - deterministico : fedavg parte dopo alcune epoche [Running] --> non serve, ora funziona as expected il 'fallback' automatico
 - probabilistic : fedavg parte con prob. inv. prop al incremento sull'accuratezza
 - open question: continuare a cambiare politica o rimanere una  volta cambiata? 

### Approaches

- Federated Learing with FedAvg: the system learns a single global model. to be considered as baseline. 
- Decentralised Learning (DecFL): Each PAIV acts both as client and parameter server of its neghbourhood. Exchange of models is one-hop.
- Distillation: 
    1. Pure Distillation: a nodes uses the neighbours' models as mixed teacher: for one class it uses the best teacher met so far.  
    2. Hybrid: Begin with Pure distillation and then it switches to DecFL when the performance on the validation set start degrading, i.e., the difference with respect to the previous result is greater than `1e-2`. The patience (how long to wait before performing the switch after the first sign of degradation) is 1. Motivation: being aggressive to avoid that the degradation process starts spreading over the network. 
    3. Self distillation (+ Hybrid): as for 1. and 2. but the nodes perform self distillation: instead of using a real teacher, they use a virtual one whose outputs are noisy (noise is my proposal, probably novel) soft labels.   
- Oracle: this is a *benchmark* and irrealistic approach: there is only one global teacher trained on the whole dataset. The nodes use it in the standard teacher-student modality. It might represent an upper bound on what a node can achieve with a KD in decentralised and heterogeneous settings using an omniscent teacher.
    

Note: 
- local training epoch for both decentralised learning and distillation lasts 30-50 epochs. Less do not provide good results. 
- all the approaches are tested on the very same data/partitioning/initial hyperparameter settings. 
### Tested on pure distillation (1.)
- comment: Distillation only gets stuck in poor local minima. PAIVS do not improve after few rounds from the beginning. It's like there is not more significant informative content in the models' exchange  . Probably because the the model circulation is "slow" (models' exchange between PAIVs is one-hop)

### Small scale Test distillation with fallback on decFL when no improvement 
- reference experiment: `hybrid-dist_dec-mnist-graph10`
- Tested tolerance 1e-2 and patience 2 (Distill with fallback)

- Comment: In this settings we observe that after 5 communication rounds the paivs behave similarly to to FedAvg and Decentralised both in train and test. As for Decentralised, there are few PAIVS that diverge w.r.t. the rest of the majority. 
#### Large scale test on 50 nodes. MNIST
- reference experiment: `hybrid-dist_dec-mnist-er_graph50` and `hybrid-dist_dec-mnist-ba_graph50`
- Tested tolerance 1e-2 and patience 2 (Distill with fallback); 10 epochs of local training (same for all the tested approaches) 
- On small scale scenario (10 nodes; mnist; connected graph) the performance of Distillation with fallback on local averaging (Dist+DecFL)are satisfacotory. 

However, on a larger scenario (50 nodes; MNIST; Barabasi-Albert graph) things slightly change. Precisely the size of the local datasets is smaller and given the non-iid distribution following a zipf with parameter 1.6, the size of hte local datasets is far smaller and, moreover, the number of patterns of the less represented classes can be no more that few tens). Therefore, we are facing a case of small local datasets. Differently from the scenarion with 10 nodes, where the size of local datasets is 5x w.r.t. the current one, peforming few local training epochs proved to be detrimental. Instead, increasing the number of local epochs to 50 (we kept the same scale factor of the dataset size reduction, but it's only an arbitrary heuristically chosen number) the performance of both DecFL and Dist-DecFL improves significantly. The latter, in particular, achieves the same performance of FedL  and eventually outperforms it. 

The interesting behaviour of adopting Distillation in the first training epochs make the Dist-DecFL approach having the same accuracy trend of FedL. However, the *performance increment saturates after few rounds* (**this is something to further investigate**), probably due to the *lack of novel information* present in the neghbourhood (**to be confirmed by experiemnts**). 
One possibility is that through the Distillation phase, the different nodes' models start getting topologically close, without suffering the disruption that comes when we do the Federated Average. 


Looking at both the DecFL and Dist-DecFL train, test and accuracy curves is appears clearly when, on average, it occurs the the nodes change learning approach, i.e., from distillation to averaging. It corresponds to the spike in train and test, and to the drop in the accuracy curve. Interestingly looking the the pure DecFL and Dist-DecFL, we can observe the very same behaviour, just shifted thanks to the initial distillation phase. 

I've been investigating if self-distillation might help with the decentralised training. The idea is substituting the "real" teachers currently covered by the peers' models with a virtual ideal teacher whose output contains random noise. Using it during the first local training improves significantly the PAIVs accuracy. 

The next question is if this self-distillation approach can be used in decentralised settings to make nodes collaborate without exchanging local models, just a summary (a statistic or a distribution) of their output. 
I tried with (i) first round with self local distillation and then (ii) virtual distillation using the the aggregated peers' outputs computed as:

- policy 1:  (sum of the probabilities) / (sum of the cardinality of each category). Preliminary results show a similr behaviour to the one of pure self distillation, i.e., it reaches a plateau after a number of communication rounds.  any improvement w.r.t. the initial self-distillation performance. The models accuracy over the communication rounds look quite stable. 

- policy 2: weighted average (of policy 1) where the weights for the p-th paiv and the l-th label is the fraction of times the label occurred during the local training. Same behaviour of 'policy 1'

- policy 3: weighted avegare noisy teacher (like in the self distillation phase). Results: No improvement

- policy 4: cherry picking of teacher's outputs using the min entropy criterium, i.e., for a given label the teacher map is populated the the peer's avg output with min entropy among those received. Results: no improvement (stuck at 75% accuracy)

Discussion (personal): apparently the use of the sole peers' outputs has limited informative power. Interestingly, from the accuracy point of view, using self-distillation and 'best-teacher-label' approaches are equivalent. However, using the models as teachers (even cherrypicked) in place of the virtual ones, looks being beneficial for the averaging phase because models start converging in the parameter space. Conversely, avereaging models after few rounds of self ditillation doesn't have the same effect, or worse, the models' parameters tend to diverge. Therefore, in the latter case, averaging models completely destroys their current configuration, as shown by the sheer accuracy drop or the sheer increment of the train and test losses. *Concluding, using only the outputs of models (average or else) as teachers in the decentralised approach does not help to transfer information between nodes*.

**All the above discussion is leading me to the idea that teh distillation approches explored so far are too "isolated", i.e., there is not enough communication between nodes. Expecially in distillation it apprears of paramount importace using a common information, which can be the same dataset. In our case the common dataset can only be nodes' local dataset. This means that we have to hose the real models for distillation OR we can start with V-KD and then pass to R-KD (or viceversa).** 

- we need to use the real models. 
- using virtual models is not enough or it is a too weak signal (probably more research is needed). 

<!-- CURRENT RUN:
- First training: pure local
- If not plateau: r-KD if there are teachers or self-KD if not.
- If plateau: social diff update. (no avg substitution) -->

**CURRENT PLAN**

1. usare oracolo senza anchoring (solo KL term) [Chiara]

2. se oracle portiamo assenza di anchoring su real-KD [Chiara]

3. rifare self-KD+FedAvg dall'inizio [Lorenzo] (no platoon) 
    Results: same initial drop as in het-decentralised but steeper growth till 96% accuracy

**Possible strategies against FedAvg Drop**
notation: 
- $w_l^t$ local model at time t
- $\{w_i^t\}_{i=1,\dots,N}$: models received from the netighbourhood. The index $i$ indicates the $i$-th device
- $\bar{w}^t = \frac{1}{N}\sum_{i=1}^N w_i^t$: avg model obtained using vanilla FedAvg
- $\Delta w^t = \|w_l^t - \bar{w}^t\|_2$: L2 norm of the distance between local model and the avg model

1. Difference-based aggregation: $w_l^{t+1} = w_l^t - \frac{(w_l^t - \bar{w}^t)}{\Delta w^t}$

**Comment on the results for policy 1.**: 
The accuracy drop problem looks mitigated. In the configuration with no-platoon, all the paivs constantly increment their performance at each communication round.  


- 3 topologie di rete: BA, RN, Stoch Block Model
- altri dataset + altri data: Fashion MNIST; FeMNIST; Synthetic
- parametri 
    - L_CE + alpha* S_KD
    - 0.7 + 0.3 rnd (beta)

Sensitivity
alpha: 
- 1 (teacher only)
- 0.5
- 0.05 DONE






